In [1]:
import sys
from sys import platform
if platform  == 'win32':
    sys.argv=["_", r"P:\rmahajan14\capstone_data\data", r"P:\rmahajan14\capstone_data\cache"]
if platform == 'linux' or platform == 'linux2':
    def full_path(p):
        return os.path.abspath(os.path.expanduser(p))
    sys.argv=["_", r"/mnt/d/DiDi_Ride_Data/data", r"/mnt/d/DiDi_Ride_Data/Ride_Data_Cache"]

In [2]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
from loader1 import *
from utils import *
from common import *
from experiment2 import *
# from sys import platform

import warnings
warnings.filterwarnings('ignore')

In [4]:
import time

Basic Coding 

In [5]:
df_final_no_decay, target_df_no_decay = get_final_df_reg(use_cache=False, decay='No Decay')

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
No Decay
../Ride_Data_Cache/active_times.msgpack exists
Decay Calculation done in 0.9689080715179443
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.21404314041137695
1f
Dumping to ../Ride_Data_Cache/hex_spatial_df.msgpack
spatial
Spatial Calculation done in 16705.16063094139


In [6]:
df_final_old_decay, target_df_old_decay = get_final_df_reg(use_cache=False, decay='Old Decay')

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
Old Decay
../Ride_Data_Cache/idle_times_old.msgpack exists
Decay Calculation done in 0.9463520050048828
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.20811223983764648
1f
../Ride_Data_Cache/hex_spatial_df.msgpack exists
spatial
Spatial Calculation done in 7.28559422492981


In [7]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay1, target_df_new_decay1 = get_final_df_reg(use_cache=False, decay='New Decay', mult_factor=1)
df_final_new_decay10, target_df_new_decay10 = get_final_df_reg(use_cache=False, decay='New Decay', mult_factor=10)
df_final_new_decay100, target_df_new_decay100 = get_final_df_reg(use_cache=False, decay='New Decay', mult_factor=100)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_1.msgpack exists
Decay Calculation done in 0.9310312271118164
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.20432305335998535
1f
../Ride_Data_Cache/hex_spatial_df.msgpack exists
spatial
Spatial Calculation done in 7.093277931213379
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_10.msgpack exists
Decay Calculation done in 0.9142119884490967
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.23170924186706543
1f
../Ride_Data_Cache/hex_spatial_df.msgpack exists
spatial
Spatial Calculation done in 6.965825080871582
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_100.msgpack exists
Decay Calculation done in 0.9041521549224854
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features 

In [8]:
df_final_new_decay10.columns

Index(['num_total_rides',  'num_pool_rides', '% of pool rides',
                    -1.0,               2.0,               4.0,
                     7.0,              12.0,              13.0,
                    14.0,
       ...
                   355.0,             356.0,             358.0,
                   360.0,             361.0,             363.0,
                   364.0,             365.0,             369.0,
         'inactive_time'],
      dtype='object', length=176)

### Modelling Code 

We have till now coded the following models 

1. Linear Regression 
2. Linear Regression with L1 Regularization
3. Linear Regression with L2 Regularization
4. Random Forest Regressor 

Now we split the data in a ratio of 75:25

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [10]:
def build_model(df, target_df, name, seed=0):
    # regression forest
    print(f'building regression model for {name}')
    X = df    
    xtrain, xtest, ytrain, ytest = train_test_split(X, target_df['target'], random_state=seed)

    sc = StandardScaler()

    xtrain_sc = sc.fit_transform(xtrain)

    rr = RandomForestRegressor(random_state=seed)
    rr.fit(xtrain_sc, ytrain)

    xtest_sc = sc.transform(xtest)
    
    print(rr.score(xtest_sc, ytest))
    
    return rr.score(xtest_sc, ytest)

In [11]:
df_list = [df_final_no_decay, df_final_old_decay, df_final_new_decay1, 
           df_final_new_decay10, df_final_new_decay100]
target_df_list = [target_df_no_decay, target_df_old_decay, target_df_new_decay1, 
                  target_df_new_decay10, target_df_new_decay100]
name_list = ['no_decay', 'old_decay', 'new_decay1', 'new_decay10', 'new_decay100']

In [12]:
result = {}
for df, target_df, name in zip(df_list, target_df_list, name_list):
    result[name] = build_model(df, target_df, name)

building regression model for no_decay
0.4192115651300975
building regression model for old_decay
0.6434035022904324
building regression model for new_decay1
0.608929696554577
building regression model for new_decay10
0.6906501087575441
building regression model for new_decay100
0.7523467893580604


In [13]:
result

{'no_decay': 0.4192115651300975,
 'old_decay': 0.6434035022904324,
 'new_decay1': 0.608929696554577,
 'new_decay10': 0.6906501087575441,
 'new_decay100': 0.7523467893580604}

In [14]:
df_final_old_decay.head()

,num_total_rides,num_pool_rides,% of pool rides,-1.0,2.0,4.0,7.0,12.0,13.0,14.0,...,355.0,356.0,358.0,360.0,361.0,363.0,364.0,365.0,369.0,inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.90122
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.99304


In [15]:
df_final_new_decay10.head()

,num_total_rides,num_pool_rides,% of pool rides,-1.0,2.0,4.0,7.0,12.0,13.0,14.0,...,355.0,356.0,358.0,360.0,361.0,363.0,364.0,365.0,369.0,inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.931860
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.210168
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,336.982874
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.410799
